# 餐飲業逾期風險預測模型
由我國「應建立食品追溯追蹤系統之食品業者」規定可知，須申報食品流向資料於本署非追不可系統多為製造與輸入業者，對於販售與餐飲業者則無相關資料可用，無法推算其庫存，進而評估其是否可能存放逾期食品。因此，針對販售與餐飲業者，則透過後市場稽查資料及業者相關資訊，嘗試以監督式學習方法建構業者逾期食品違規風險預測模型，而此處為餐飲業資料的建模評估。

### 程式架構
* **資料前處理(Preprocessing.ipynb)**
* 特徵篩選(ModelFitting.ipynb)
* 模型配適(FeatureSelection.ipynb)

## 資料前處理

### Package

In [1]:
import numpy as np 
import pandas as pd

### Data Import
原始資料從資料庫擷取至excel，存於3張工作表：
* Factor：業者特徵，來自PMDS及非登不可。
* ExpCase：逾期食品違規案件，從GHP違規案件中尋找。
* WebExpCase：「逾期食品稽查案件」中違規案件，由於逾期食品稽查自2024年才分為獨立項目，故另從系統匯出。

In [2]:
dict_dfs_ori = pd.read_excel('..\data\data_ori.xlsx', sheet_name=None)    # get all worksheets
print(dict_dfs_ori.keys())

dict_keys(['Factor', 'ExpCase', 'WebExpCase'])


In [3]:
# 違規案件稽查編號
set_expSN = set(dict_dfs_ori['ExpCase']['稽查事件編號']) | set(dict_dfs_ori['WebExpCase']['稽查序號'])

# 建立完整資料表，含 target, features, others
df_ori = dict_dfs_ori['Factor'].copy()
df_ori['target'] = df_ori['稽查事件編號'].isin(set_expSN).astype('uint8')    # 透過比對案件編號是否在違規案件中

In [4]:
del dict_dfs_ori, set_expSN

### Data Preprocess
先將資料縱切 3 份：目標(target)、特徵(features)、其它(others)，針對特徵做前處理。

In [5]:
print(df_ori.columns)

Index(['稽查事件編號', '營業縣市', '是否兼具製造業', '是否兼具輸入業', '是否兼具販售業', '是否兼具物流業', '餐飲業次業別數',
       '上市上櫃', '網路銷售', '導入HACCP', '導入ISO22000', '來源流向管理', '投保責任險', '倉儲', '實驗室',
       '標章分數', '資本額', '食添法規認知', '逾期食品法規認知', '有直營連鎖店', '有加盟連鎖店', '同一地址有多個餐飲場所',
       '有餐飲場所且有工廠製造場所', '有餐飲場所且委託他廠代工', '美食平台', '有無專技人員', '場所型式', '場所規模',
       '有無技術士證照', '有無供應學校', 'Group', 'target'],
      dtype='object')


In [6]:
cols_notFeat = ['稽查事件編號', 'Group', 'target']

ser_target = df_ori[cols_notFeat[-1]]
df_features = df_ori.drop(cols_notFeat, axis=1)    # inplace = False (default), return a copy
df_others = df_ori[cols_notFeat[:-1]]

In [7]:
del df_ori, cols_notFeat

##### 缺失值
<div>檢視特徵有無缺失值，以下列程式碼快速判斷：</div>
<ol>
    <li>isna()/isnull()：檢查資料表每一個值是否為缺失值，輸出Dataframe。</li>
    <li>any(axis=0)：檢查每一欄是否有True (reduce the index)，輸出Series。</li>
    <li>sum()：值加總。</li>
</ol>
<div>等於0表示無缺失值，反之。</div>

In [8]:
print(df_features.isna().any(axis=0).sum())

0


##### 資料型別轉換
<div>以pandas讀取excel資料若不設定欄位資料型別，數值欄位會被視作int64或float64，字串欄位會被視作object。</div>
<div>此處進行特徵資料型別轉換，包含3種型別：</div>
<ol>
    <li>無序類別，用category。</li>
    <li>有序類別，用category並設定ordered=True。</li>
    <li>布林，用uint8，考量建模相容性。</li>
    <li>數值，用uint32，僅資本額。</li>
</ol>

In [9]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer

from pandas.api.types import CategoricalDtype    # 自行建立 ordered category
cat_ord = CategoricalDtype(categories=None, ordered=True)    # 不設類別 (default) ，有序

In [10]:
# 各型別特徵名稱
cols_to_int = ['資本額', ]
cols_to_cat = ['營業縣市', '場所型式', ]
cols_to_ord = ['餐飲業次業別數', '標章分數', '場所規模', ]

# 轉換函數
to_int = FunctionTransformer(lambda x: x.astype('uint32'))    # uint32: 0 to (2**32)-1
to_cat = FunctionTransformer(lambda x: x.astype('category'))
to_ord = FunctionTransformer(lambda x: x.astype(cat_ord))

# 轉換流程
col_trans = make_column_transformer(
    (to_int, cols_to_int), (to_cat, cols_to_cat), (to_ord, cols_to_ord), 
    remainder='drop',    # 丟棄剩餘欄位 (default)
    verbose_feature_names_out=False    # 去掉 transformer 前綴
).set_output(transform='pandas')    # 輸出 dataframe

# 執行轉換
df_features_nc = col_trans.fit_transform(df_features)


In [11]:
from pandas.api.types import is_integer_dtype    # 檢測欄位是否為 int

In [12]:
# 保留二元特徵
df_features_b = df_features.drop(labels=df_features_nc.columns, axis=1)

# 轉換至 0/1 ，型別設定 uint8
df_features_b = df_features_b.apply(    # axis=0: apply function to each column
    # use conditional expression
    lambda col: col.astype('uint8') if is_integer_dtype(col) else col.map({'Y':1, 'N':0}).astype('uint8')
)

In [13]:
# concatenate dataframe
df_features_ncb = pd.concat([df_features_nc, df_features_b], axis=1)

In [14]:
del cat_ord, cols_to_int, cols_to_cat, cols_to_ord, to_int, to_cat, to_ord, col_trans, df_features_nc, df_features_b

### Splitting Datasets
切分訓練集與測試集，測試集不得用於後續特徵處理及建模。

In [15]:
# training set
X_train = df_features_ncb[df_others['Group'] == 'train']
y_train = ser_target[df_others['Group'] == 'train']

# testing set
X_test = df_features_ncb[df_others['Group'] == 'test']
y_test = ser_target[df_others['Group'] == 'test']

del df_features, ser_target, df_features_ncb

In [16]:
# pickle object to file
dict_train = {'X': X_train, 'y': y_train}
dict_test = {'X': X_test, 'y': y_test}
dict_dfs_pre = {'others': df_others, 'train': dict_train, 'test': dict_test}
# 出現 nested dictionary ，注意複製問題！

# pd.to_pickle(dict_dfs_pre, '../data/data_pre.pkl')